 **GLOBAL PUBLIC HEALTH INDICATORS**

WHY WE CHOSE THIS?
* health is the most fundamental measure of a society’s well-being.
* huge inequality still exists between countries.
* understanding disease patterns shows where the world is progressing and where it is failing.
* many diseases are preventable, yet millions still die.
* data shows truths that governments often hide or underplay.
* health outcomes reflect social justice.



DATA PREPROCESSING
* clean datasets
* same year align
* missing values removed
* only valid countries kept
* handled GDP outliers


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
life = pd.read_csv("/kaggle/input/global-public-health-indicators/life-expectancy.csv")
infant = pd.read_csv("/kaggle/input/global-public-health-indicators/child-mortality.csv")
gdp = pd.read_csv("/kaggle/input/global-public-health-indicators/gdp-per-capita-maddison-project-database.csv")
regions=pd.read_csv("/kaggle/input/global-public-health-indicators/continents-according-to-our-world-in-data.csv")
gdp=gdp.drop(columns=["900793-annotations"])
gdp.isna().sum()
# life.isna().sum()
# infant.isna().sum()
gdp.head(100)
regions.isna().sum()
gdp
life
infant



In [ ]:
# Life expectancy
life_countries = life[life['Code'].notna()].reset_index(drop=True)

# Infant mortality
infant_countries = infant[infant['Code'].notna()].reset_index(drop=True)

#GDP per Capita
gdp_countries = gdp[gdp['Code'].notna()].reset_index(drop=True)

#for regions

regions = regions.rename(columns={
    "World regions according to OWID": "Region"
})

for df in [life_countries,infant_countries,gdp_countries]:
    df.drop(columns=["Code"],inplace=True)



In [ ]:
infant_countries

**QUES1:  How strongly do income levels correlate with life expectancy and infant mortality across regions? Are there outlier countries that perform better or worse than expected given their income?**


In [ ]:
country_data = gdp_countries.merge(life_countries, on=['Entity','Year'])
country_data = country_data.merge(infant_countries, on=['Entity','Year'])

regional_data = country_data.merge(regions[['Entity','Region']], on='Entity', how='left')


In [ ]:
regional_avg = regional_data.groupby(['Region']).agg({    'GDP per capita': 'median',
    'Period life expectancy at birth': 'median',
    'Child mortality rate': 'median'
}).reset_index()

country_avg=country_data.groupby(['Entity']).agg({
 'GDP per capita': 'mean',
    'Period life expectancy at birth': 'mean',
    'Child mortality rate': 'mean'
}).reset_index()


region_corr = regional_avg[['GDP per capita',
                           'Period life expectancy at birth',
                           'Child mortality rate']].corr()
print(region_corr)

| Pairs                             | Correlation | Meaning                                |
|----------------------------------|-------------|----------------------------------------|
| GDP per capita ↔ Life Expectancy  | +0.74       | Higher income → longer lifespan        |
| GDP per capita ↔ Child Mortality  | −0.71       | Higher income → lower child mortality  |
| Life Expectancy ↔ Child Mortality | −0.99       | Very strong inverse relationship       |


**GDP per Capita vs Life Expectancy → Strong positive correlation****

As GDP increases, countries invest more in:

* hospitals, doctors, infrastructure

* medicines and technology

* sanitation and clean water
  >      People live longer.

**GDP per Capita vs Child Mortality → Strong negative correlation**

Richer countries provide:

* better maternal healthcare

* good nutrition for children

* strong vaccination programs
    >  Child deaths drastically reduce.

**Life Expectancy vs Child Mortality → Very strong negative correlation (−0.996)**

Extremely high dependency:

* If child mortality falls → life expectancy shoots up

* If child mortality rises → life expectancy collapses
      Child health defines overall population survival.

In [ ]:

plt.figure(figsize=(8,5))
sns.scatterplot(
    data=regional_avg,
    x='GDP per capita',
    y='Period life expectancy at birth',
    hue='Region'
)
plt.xscale('log')
plt.title("Region-wise: GDP vs Life Expectancy")
plt.show()


**Region-wise: GDP vs Life Expectancy**

**Insight**

Higher GDP regions like Oceania, Europe & North America have the highest life expectancy (70–75 years).

Africa, with the lowest GDP, has life expectancy around 53 years — the lowest among all regions.

Asia & South America fall in the middle cluster, showing improving healthcare outcomes with increasing income.

> This shows a direct positive correlation:

Higher GDP → Better healthcare → Longer life expectancy

In [ ]:


plt.figure(figsize=(8,5))
sns.scatterplot(
    data=regional_avg,
    x='GDP per capita',
    y='Child mortality rate',
    hue='Region'
)
plt.xscale('log')
plt.title("Region-wise: GDP vs Infant Mortality")
plt.show()



**Region-wise: GDP vs Child Mortality Rate**

**Insight**

Child mortality rate drops drastically with increasing GDP.

Africa has the highest child mortality despite low GDP.

Europe & Oceania show the lowest mortality levels, reflecting strong maternal & child healthcare facilities.

> The correlation here is strongly negative:

Higher GDP → Lower child mortality

In [ ]:

plt.figure(figsize=(8,5))
sns.scatterplot(
    data=country_avg,
    x='GDP per capita',
    y='Period life expectancy at birth',
    hue='Entity'
)
plt.xscale('log')
plt.title("country-wise: GDP vs Life Expectancy")
plt.legend([],[], frameon=False)
plt.show()


**Country-wise: GDP vs Life Expectancy**

**Insight**

The scatter is more widely spread compared to region-level plot → inequality across countries.

High-income countries cluster around 70–80 years, but…

Some countries with a moderately high GDP enjoy very good life expectancy, meaning strong public health policy.

In [ ]:

plt.figure(figsize=(8,5))
sns.scatterplot(
    data=country_avg,
    x='GDP per capita',
    y='Child mortality rate',
    hue='Entity'
)
plt.xscale('log')
plt.title("country-wise: GDP vs Child mortality rate")
plt.legend([],[], frameon=False)
plt.show()

**Country-wise: GDP vs Child Mortality Rate**

**Insight**

Child mortality still remains lower in some low-GDP countries; hence, targeted health interventions remain effective.

While a few countries with moderate GDP still have inequality in resource distribution.


In [ ]:
country_avg.sort_values(by=["Child mortality rate"], ascending=False).tail(5)
country_avg.sort_values(by=["Child mortality rate"], ascending=False).head(5)

In [ ]:
country_avg.sort_values(by=["Period life expectancy at birth"], ascending=False).head(5)
country_avg.sort_values(by=["Period life expectancy at birth"], ascending=False).tail(5)

In [ ]:
df_grouped = country_avg.copy()

df_grouped["gdp_group"] = pd.qcut(df_grouped["GDP per capita"], q=5, labels=[
    "Very Low Income",
    "Low Income",
    "Middle Income",
    "Upper Middle Income",
    "High Income"
])
group_stats = df_grouped.groupby("gdp_group", observed=False).agg({
    "Period life expectancy at birth": "mean",
    "Child mortality rate": "mean"
}).reset_index()

df_grouped = df_grouped.merge(group_stats, on="gdp_group", suffixes=("", "_expected"))

df_grouped["life_diff"] = df_grouped["Period life expectancy at birth"] - df_grouped["Period life expectancy at birth_expected"]
df_grouped["child_diff"] = df_grouped["Child mortality rate_expected"] - df_grouped["Child mortality rate"]  
# child_diff positive = better than expected
df_grouped["health_score"] = df_grouped["life_diff"] + df_grouped["child_diff"]
positive_outliers = df_grouped.sort_values("health_score", ascending=False).head(10)
negative_outliers = df_grouped.sort_values("health_score").head(10)
positive_outliers
# negative_outliers

**Positive Outliers (better health outcomes than expected)**

Examples
North Korea, Cuba, Dominica, Palestine, Sri Lanka

These countries are overperforming relative to their income → strong community health systems.

**Negative Outliers (worse health outcomes than expected)**

Examples
Equatorial Guinea, Liberia, Nigeria, Sweden, France

These countries show policy gaps or unequal healthcare access, despite economic advantages.

**QUES2:Which diseases contribute most to mortality globally, and how does this vary by region (e.g., infectious diseases in developing nations vs. chronic diseases in developed nations)?**

In [ ]:
death_1980_1993 = pd.read_csv("/kaggle/input/global-public-health-indicators/IHME-GBD_2023_DATA-e78625dc-1.csv")
death_1994_2022 = pd.read_csv("/kaggle/input/global-public-health-indicators/IHME-GBD_2023_DATA-b36b69f4-1.csv")
specific_cause_data=pd.read_csv("/kaggle/input/global-public-health-indicators/IHME-GBD_2023_DATA-71172ff2-1.csv")
cause_of_death_data = pd.concat(
    [death_1980_1993, death_1994_2022,specific_cause_data],
    ignore_index=True
)
cause_of_death_data = cause_of_death_data.rename(columns={
    'location': 'Entity',
    'year':'Year'
})

country_filtered = country_data[
    (country_data['Year'] >= 1980) & (country_data['Year'] <= 2022)
]

print(country_filtered.columns)
print(cause_of_death_data.columns)
cause_of_death_data
final_merged_data = pd.merge(
    country_filtered,
    cause_of_death_data,
    on=['Entity', 'Year'],
    how='inner'
)

final_merged_data

In [ ]:
df_deaths = final_merged_data[
    (final_merged_data['measure'] == 'Deaths') &
    (final_merged_data['metric'] == 'Number')
]

global_top_causes = (
    df_deaths.groupby("cause")["val"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

global_top_causes


In [ ]:
df = final_merged_data[
    (final_merged_data["measure"] == "Deaths") &
    (final_merged_data["metric"] == "Number")
]
df2 = df[df["cause"] != "All causes"]

yearly_cause = (
    df2.groupby(["Year", "cause"])["val"]
       .sum()
       .reset_index()
)

yearly_top = (
    yearly_cause.sort_values(["Year", "val"], ascending=[True, False])
                .groupby("Year")
                .head(1)
)

yearly_top

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(yearly_top["Year"], yearly_top["val"], marker='o')
plt.title("Global Deaths by the Deadliest Disease Each Year (1980–2022)")
plt.xlabel("Year")
plt.ylabel("Deaths")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14,6))
plt.scatter(yearly_top["Year"], yearly_top["cause"])
plt.plot(yearly_top["Year"], yearly_top["cause"], alpha=0.4)
plt.title("Deadliest Disease Globally Each Year (1980–2022)")
plt.xlabel("Year")
plt.ylabel("Disease")
plt.grid(True)
plt.tight_layout()
plt.show()


##  Here Overall Trend Summary

| Period      | Deadliest Disease          | Why                                      |
|-------------|-----------------------------|-------------------------------------------|
| 1980–2004   | Stroke                      | High global hypertension, aging population |
| 2005–2020   | Ischemic Heart Disease (IHD)| Lifestyle diseases dominate                |
| 2021        | COVID-19                    | Pandemic mortality spike                  |
| 2022        | Ischemic Heart Disease (IHD)| COVID decline, chronic disease dominance   |


**Conclusion**

Stroke dominated global deaths from 1980 to the early 2000s. However, from 2005 onward, Ischemic Heart Disease overtook Stroke and became the leading global cause of death, reflecting the worldwide shift toward chronic, lifestyle-driven diseases. In 2021, COVID-19 temporarily became the deadliest cause globally, but by 2022, IHD returned as the top killer. This aligns perfectly with official IHME global disease burden trends.

In [ ]:
final_df = final_merged_data.merge(
    df_grouped[["Entity", "gdp_group"]],
    on="Entity",
    how="left"
)
cause_by_gdp = (
    final_df[
        (final_df["measure"] == "Deaths") &
        (final_df["metric"] == "Number") &
        (final_df["cause"] != "All causes")
    ]
    .groupby(["gdp_group", "cause"], observed=False)["val"]
    .sum()
    .reset_index()
)

top5_by_group = (
    cause_by_gdp.sort_values(["gdp_group", "val"], ascending=[True, False])
                .groupby("gdp_group", observed=False)
                .head(5)
)
top5_by_group


In [ ]:

top5_by_group["gdp_group"] = top5_by_group["gdp_group"].astype("category")
plt.figure(figsize=(14,8))
sns.barplot(
    data=top5_by_group,
    x="gdp_group",
    y="val",
    hue="cause",
    errorbar=None
)

plt.title("Top 5 Deadliest Diseases by Income Group")
plt.xlabel("Income Group")
plt.ylabel("Total Deaths (1980–2022)")
plt.xticks(rotation=45)
plt.legend(title="Cause", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()




### **(Based on GDP-based income categories)**


#  Very Low-Income Countries (Poorest Nations) 
### **Top Deadliest Diseases:**
- **HIV/AIDS**  
- **Malaria**  
- **Stroke**  
- **Tuberculosis**  
- **Measles**

### **Interpretation:**
- High burden of **infectious diseases**  
- Poor sanitation and low vaccination coverage  
- Weak healthcare systems  
- High child mortality  
- Lack of access to HIV/TB treatment  
- Stroke appears due to **untreated hypertension**

---

# Low-Income Countries
### **Top Deadliest Diseases:**
- **Ischemic heart disease (IHD)**  
- **Stroke**  
- **Tuberculosis**  
- **Diabetes mellitus**  
- **Measles**

### **Interpretation:**
- These nations are in a **transition stage**  
- Infectious diseases still significant  
- Chronic diseases rising due to urbanization and lifestyle changes  
- Shift begins from **communicable → non-communicable diseases (NCDs)**

---

#   Middle-Income Countries
### **Top Deadliest Diseases:**
- **Stroke**  
- **Ischemic heart disease**  
- **Alzheimer’s disease & dementias**  
- **Diabetes mellitus**  
- **Tuberculosis**

### **Interpretation:**
- A **dual burden** of infectious + chronic diseases  
- Rapid lifestyle modernization → stroke, diabetes rise  
- Aging population → dementia increases  
- TB still not eliminated

---

#   Upper-Middle Income Countries 
### **Top Deadliest Diseases:**
- **Ischemic heart disease**  
- **Stroke**  
- **Alzheimer’s disease & dementias**  
- **Diabetes mellitus**  
- **Breast cancer**

### **Interpretation:**
- Disease profile similar to developed countries  
- Very low infectious disease mortality  
- Chronic diseases dominate  
- Cancer becomes prominent due to longer lifespan

---

#   High-Income Countries (Rich Nations) 
### **Top Deadliest Diseases:**
- **Ischemic heart disease**  
- **Stroke**  
- **Alzheimer’s & dementia**  
- **Diabetes mellitus**  
- **Breast cancer**

### **Interpretation:**
- **Fully chronic disease profile**  
- Aging populations → dementia + heart disease top causes  
- Lifestyle risks → diabetes, cancer  
- Infectious diseases extremely low due to strong healthcare

---




**Pattern Found:**

As income increases → Infectious diseases drop

As income increases → Chronic diseases rise

### Q3. Are higher COVID-19 vaccination rates associated with declines in COVID-19 cases & deaths?
Why this question?

Vaccination is one of the strongest public-health tools.

Analysing vaccination vs disease spread shows real-world impact.

Countries adopted vaccines at different speeds → useful inequality insights.

In [ ]:
vax = pd.read_csv("/kaggle/input/global-public-health-indicators/daily-covid-19-vaccine-doses-administered-per-million-people.csv")
cases = pd.read_csv("/kaggle/input/global-public-health-indicators/daily-new-confirmed-covid-19-cases-per-million-people.csv")
deaths = pd.read_csv("/kaggle/input/global-public-health-indicators/daily-new-confirmed-covid-19-deaths-per-million-people.csv")

print("Loaded datasets:")
print("Vaccination:", vax.shape)
print("Cases:", cases.shape)
print("Deaths:", deaths.shape)


In [ ]:
for df in [vax, cases, deaths]:
    df.rename(columns={"Entity": "Country", "Day": "date"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"])

# Merge datasets
covid = (
    cases.merge(vax, on=["Country", "date"], how="left")
         .merge(deaths, on=["Country", "date"], how="left")
)
covid["COVID-19 doses (daily, 7-day average, per million people)"] = covid["COVID-19 doses (daily, 7-day average, per million people)"].fillna(0)

# Sort properly for time operations
covid = covid.sort_values(["Country", "date"]).reset_index(drop=True)

# Filter to show only 2021 onwards (when vaccination actually started)
covid = covid[covid["date"] >= "2021-01-01"].reset_index(drop=True)

covid.head(20)


In [ ]:
# --- SHELL 3: DETECT COLUMNS + CLEAN ZERO VALUES -----

print("Columns:", covid.columns.tolist())

# Auto-detect correct column names
case_col  = [c for c in covid.columns if "case" in c.lower()][0]
death_col = [c for c in covid.columns if "death" in c.lower()][0]
vax_col   = [c for c in covid.columns if ("vaccine" in c.lower() or "dose" in c.lower())][0]

print("\nDetected columns:")
print("Cases:", case_col)
print("Deaths:", death_col)
print("Vaccination:", vax_col)

# Replace NaN with 0
covid[case_col]  = covid[case_col].fillna(0)
covid[death_col] = covid[death_col].fillna(0)
covid[vax_col]   = covid[vax_col].fillna(0)

# KEEP ONLY ROWS WHERE VACCINATION > 0 OR CASES/DEATHS > 0
covid = covid[(covid[vax_col] > 0) | (covid[case_col] > 0) | (covid[death_col] > 0)]

# 7-day rolling averages
covid["cases_7d"] = covid.groupby("Country")[case_col].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)

covid["deaths_7d"] = covid.groupby("Country")[death_col].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)

# Cumulative vaccination
covid["vax_cumsum"] = covid.groupby("Country")[vax_col].cumsum()

covid.head(20)


In [ ]:
country = "India"   # you can change to any country

temp = covid[covid["Country"] == country]

plt.figure(figsize=(12,5))
plt.plot(temp["date"], temp["cases_7d"], label="Cases (7-day average)")
plt.plot(temp["date"], temp["vax_cumsum"]/100, label="Cumulative Vaccination (/100 people)", color="green")

plt.title(f"COVID-19 Trend — {country}")
plt.xlabel("Date")
plt.ylabel("Values")
plt.legend()
plt.show()


In [ ]:
def lag_corr(df, country, lag=21):
    c = df[df["Country"] == country].copy()
    if c.shape[0] < lag + 5:
        return np.nan
    c["vax_lag"] = c["vax_cumsum"].shift(lag)
    c = c.dropna(subset=["cases_7d", "vax_lag"])
    if c["cases_7d"].nunique() <= 1 or c["vax_lag"].nunique() <= 1:
        return np.nan
    return c["cases_7d"].corr(c["vax_lag"])

# Compute lag correlations for all countries
lag_list = []
for c in covid["Country"].unique():
    lag_list.append([c, lag_corr(covid, c, lag=21)])  # vaccine effect after ~21 days

lag_df = pd.DataFrame(lag_list, columns=["Country", "lag_corr"])
lag_df.sort_values("lag_corr").head()


In [ ]:
print("INTERPRETATION")
print("--------------")
print("Negative correlation means:")
print(" Higher vaccination → lower cases after ~21 days.\n")

print("Top 10 countries showing the strongest vaccine impact:")
display(lag_df.nsmallest(10, "lag_corr"))

print("\nAverage global lag-correlation:")
print(lag_df["lag_corr"].mean())


# 📝 Q4. Do developing countries show faster improvements in health outcomes over time compared to developed ones, or is the gap widening?

### Why this question?

Global health inequality remains one of the biggest challenges.

Developing countries often face limited access to healthcare, nutrition, vaccines, and sanitation.

Analysing long-term trends helps us understand:

- Which countries are improving fastest  
- Whether the health gap is narrowing or widening  
- How income levels influence life expectancy & mortality  

Health outcomes reflect:

- Social justice  
- Economic development  
- Healthcare access  
- Policy effectiveness  


In [ ]:
life = pd.read_csv("/kaggle/input/global-public-health-indicators/life-expectancy.csv")
child = pd.read_csv("/kaggle/input/global-public-health-indicators/child-mortality.csv")
gdp = pd.read_csv("/kaggle/input/global-public-health-indicators/gdp-per-capita-maddison-project-database.csv")
regions = pd.read_csv("/kaggle/input/global-public-health-indicators/continents-according-to-our-world-in-data.csv")

print("Loaded datasets:")
print("Life expectancy:", life.shape)
print("Child mortality:", child.shape)
print("GDP:", gdp.shape)
print("Regions:", regions.shape)

### 🧹 Data Cleaning & Standardization

Before analysing trends, we need to:
- Keep only valid country entries
- Standardize column names across datasets
- Prepare GDP, life expectancy, and mortality for merging

This ensures the merged dataset is accurate and consistent.


### 🔗 Merging All Datasets Into One Master Table

We now merge:
- Life expectancy
- Child mortality
- GDP per capita
- Region information

This gives us a single dataframe containing **all health indicators per country per year**.

we already have regional_data

In [ ]:
regional_data

###  Creating Income Groups (Developed vs Developing)

To compare countries fairly:
- We classify countries into **5 GDP-based income groups**
- Countries in the highest group are labeled **Developed**
- Remaining groups are labeled **Developing**

This allows us to study long-term health inequality.

In [ ]:
df=regional_data.copy()
df["income_group"] = pd.qcut(
    df["GDP per capita"],
    q=5,
    labels=["Low", "Lower-Middle", "Middle", "Upper-Middle", "High"]
)

# Developed vs Developing classification
df["dev_status"] = df["income_group"].apply(
    lambda x: "Developed" if x == "High" else "Developing"
)

df["dev_status"].value_counts()

In [ ]:
# Compute mean values per year for each development category
trend = df.groupby(["Year", "dev_status"]).agg({
    "Period life expectancy at birth": "mean",
    "Child mortality rate": "mean"
}).reset_index()

# Life Expectancy trend
plt.figure(figsize=(10,5))
sns.lineplot(data=trend, x="Year", y="Period life expectancy at birth", hue="dev_status")
plt.title("Life Expectancy Trend (Developed vs Developing)")
plt.xlabel("Year")
plt.ylabel("Average Life Expectancy")
plt.show()

# Child mortality trend
plt.figure(figsize=(10,5))
sns.lineplot(data=trend, x="Year", y="Child mortality rate", hue="dev_status")
plt.title("Child Mortality Trend (Developed vs Developing)")
plt.xlabel("Year")
plt.ylabel("Child Mortality Rate")
plt.show()

In [ ]:
latest_year = trend["Year"].max()
latest = trend[trend["Year"] == latest_year]

print(" FINAL YEAR COMPARISON:")
print(latest)

print("\nInsights:")
print("- Developing countries have improved life expectancy steadily, often faster due to starting from lower baselines.")
print("- Child mortality has dropped significantly in developing countries, showing major health improvements.")
print("- However, developed nations still maintain higher life expectancy and lower mortality → the gap is narrowing but not closed.")
print("- Overall: Developing countries are catching up, but inequality still exists.")

**QUE5: How do healthcare spending and government policy indicators explain differences in health outcomes?**

In [ ]:
health_exp = pd.read_csv("/kaggle/input/global-public-health-indicators/annual-healthcare-expenditure-per-capita.csv")
health_exp = health_exp[["Entity", "Year", "Current health expenditure per capita, PPP (current international $)"]]
health_exp.columns = ["Entity", "Year", "healthcare_spending"]

In [ ]:
df_last = life.merge(child, on=["Entity","Year"], how="inner")
df_last = df_last.merge(health_exp, on=["Entity","Year"], how="inner")
df_last = df_last.dropna()
df_last

In [ ]:
df_last[['Period life expectancy at birth', 'Child mortality rate', 'healthcare_spending']].corr()


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_last["healthcare_spending"], df_last["Period life expectancy at birth"])
plt.xscale("log")
plt.xlabel("Healthcare Spending per Capita (USD, log scale)")
plt.ylabel("Life Expectancy (Years)")
plt.title("Healthcare Spending vs Life Expectancy (Log Scale)")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_last['healthcare_spending'], df_last['Child mortality rate'])
plt.xscale("log") 
plt.xlabel("Healthcare Spending per Capita (USD)")
plt.ylabel("Infant Mortality Rate")
plt.title("Healthcare Spending vs Infant Mortality")
plt.show()

**CONCLUSION**


**Spending ↑ → Life Expectancy ↑**

The scatter plot with life expectancy shows a strong upward trend.
As healthcare spending increases, people tend to live longer.
This is because stronger health systems mean:

* better hospitals

* more doctors

* easier access to treatment

* good vaccination and prevention programs

Countries like Japan, Norway, and the US have high spending and also very high life expectancy.

**Spending ↑ → Child Mortality ↓**

The second plot (health spending vs infant mortality) shows the exact opposite trend.
Higher spending leads to very low child death rates.

Poor countries with low spending have higher child mortality because:

* fewer newborn care facilities

* weaker vaccination coverage

* shortage of medical staff

* poor nutrition

Rich countries spend more → fewer children die young.

In [ ]:
countries = ["India", "United States", "Japan", "Norway"]

for c in countries:
    temp = df_last[df_last['Entity'] == c]
    plt.plot(temp['Year'], temp["Period life expectancy at birth"], label=c)

plt.legend()
plt.xlabel("Year")
plt.ylabel("Life Expectancy")
plt.title("Life Expectancy Over Time – Country Comparison")
plt.show()


**Comparing Countries Over Time**

The line chart comparing India, US, Japan, Norway shows that:

* All countries improved over time

* But rich countries always stayed ahead

* India improved the fastest because it started from a lower point

* Japan and Norway stay at the top with very high life expectancy

* The US spends a lot but still doesn’t beat Japan or Norway (this hints at policy issues, inequality, etc.)